In [2]:
import cirq
import numpy as np

In [3]:
def int_to_Gray(base_10_num, n_qubits):
    # https://en.wikipedia.org/wiki/Gray_code

    # print(np.binary_repr(num, n_qubits)) # standard binary form!

    # The operator >> is shift right. The operator ^ is exclusive or
    gray_int = base_10_num ^ (base_10_num >> 1)

    return np.binary_repr(gray_int, n_qubits)

In [14]:
n_qubits=4

state_array = np.arange(0,2**n_qubits,1)
print(state_array)

grey_state=np.array([int_to_Gray(i, n_qubits) for i in state_array])
print(grey_state)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
['0000' '0001' '0011' '0010' '0110' '0111' '0101' '0100' '1100' '1101'
 '1111' '1110' '1010' '1011' '1001' '1000']


In [35]:
start_index = np.where(grey_state==''.join(['1' for _ in range(n_qubits)]))[0][0]

In [42]:
np.hstack((grey_state[start_index::], grey_state[:start_index]))

array(['1111', '1110', '1010', '1011', '1001', '1000', '0000', '0001',
       '0011', '0010', '0110', '0111', '0101', '0100', '1100', '1101'],
      dtype='<U4')

In [636]:
def control_circuit(n_ancilla_qubits): # list_system_qubits
    ancilla_qubits = list(cirq.LineQubit.range(n_ancilla_qubits))
    
    n_work_qubits = n_ancilla_qubits-1
    work_qubits = list(cirq.LineQubit.range(n_ancilla_qubits, n_work_qubits+n_ancilla_qubits))
    
    system_qubit = cirq.LineQubit(n_work_qubits+n_ancilla_qubits) #need to change this when doing properly!
    
    state_array = np.arange(0,2**n_ancilla_qubits,1)
    grey_state_array =np.array([int_to_Gray(i, n_ancilla_qubits) for i in state_array])
    
    start_index = np.where(grey_state_array==''.join(['1' for _ in range(n_ancilla_qubits)]))[0][0] # find all '11...1' state
    
    control_states_in_order = np.hstack((grey_state_array[start_index::], grey_state_array[:start_index]))
    
    
    control_circuit_list=[]
    for control_str in control_states_in_order:
        control_int_list = [int(bit) for bit in control_str]
#         print(control_int_list)
        circuit_list=[]
        for index, work_bit in enumerate(work_qubits):
            if index ==0:
                circuit_list.append(cirq.Circuit(cirq.X.controlled(num_controls=2,
                                               control_values=control_int_list[:2]
                                              ).on(*ancilla_qubits[:2], work_bit)))
                
            elif index<n_work_qubits-1:
                circuit_list.append(cirq.Circuit(cirq.X.controlled(num_controls=2,
                                               control_values= [control_int_list[1+index], 1] 
                                              ).on(ancilla_qubits[1+index], work_qubits[index-1], work_qubits[index])))
            else:
                circuit_list.append(cirq.Circuit(cirq.X.controlled(num_controls=2,
                                               control_values= [control_int_list[1+index], 1] 
                                              ).on(ancilla_qubits[1+index], work_qubits[index-1], work_qubits[index])))
        
        control_P = cirq.Circuit(cirq.Y.controlled(num_controls=1,
                                               control_values= [1] 
                                              ).on(work_qubits[-1], system_qubit))
        
        full_circuit=[circuit_list, control_P, circuit_list[::-1]]
        
#         print(cirq.Circuit(full_circuit))
#         print('')
        control_circuit_list.append(cirq.Circuit(full_circuit))
        
    return control_circuit_list
        

unsimplified_gate_seq = control_circuit(4)

In [637]:
unsimplified_gate_seq[0]

0: ───@───────────────────────@───
      │                       │
1: ───@───────────────────────@───
      │                       │
2: ───┼───@───────────────@───┼───
      │   │               │   │
3: ───┼───┼───@───────@───┼───┼───
      │   │   │       │   │   │
4: ───X───@───┼───────┼───@───X───
          │   │       │   │
5: ───────X───@───────@───X───────
              │       │
6: ───────────X───@───X───────────
                  │
7: ───────────────Y───────────────

In [638]:
list(unsimplified_gate_seq[0].all_operations())[-1].gate
# list(unsimplified_gate_seq[0].all_operations())[-1].gate.sub_gate
# list(unsimplified_gate_seq[0].all_operations())[-1].qubits
# list(unsimplified_gate_seq[0].all_operations())[-1].control_values

cirq.ControlledGate(sub_gate=cirq.X, num_controls=2)

In [639]:
circuit_1_ops = list(unsimplified_gate_seq[0].all_operations())
circuit_2_ops = list(unsimplified_gate_seq[1].all_operations())
len(circuit_1_ops[0].control_values)

2

In [640]:
def recursive_simplification(circuit_1_ops, circuit_2_ops):
    if (circuit_1_ops[-1] == circuit_2_ops[0]) and (len(circuit_1_ops[-1].control_values)==2):
        circuit_1_ops.pop(-1)
        circuit_2_ops.pop(0)
        
        return recursive_simplification(circuit_1_ops, circuit_2_ops)
    else:
        return circuit_1_ops, circuit_2_ops
        
# circuit_1_ops = list(unsimplified_gate_seq[0].all_operations())
# circuit_2_ops = list(unsimplified_gate_seq[1].all_operations())
# c1, c2 = recursive_simplification(circuit_1_ops, circuit_2_ops)
# print(cirq.Circuit(c1))
# print(cirq.Circuit(c2))

In [641]:
simple_gate_cancel_list=unsimplified_gate_seq.copy()

for index in range(len(simple_gate_cancel_list)-1):
    circuit_1_ops = list(simple_gate_cancel_list[index].all_operations())
    circuit_2_ops = list(simple_gate_cancel_list[index+1].all_operations())
    c1_new, c2_new = recursive_simplification(circuit_1_ops, circuit_2_ops)
                                                                                               
    simple_gate_cancel_list[index] = cirq.Circuit(c1_new)
    simple_gate_cancel_list[index+1] = cirq.Circuit(c2_new)
    
simple_gate_cancel_list

[0: ───@───────────────────
      │
1: ───@───────────────────
      │
2: ───┼───@───────────────
      │   │
3: ───┼───┼───@───────@───
      │   │   │       │
4: ───X───@───┼───────┼───
          │   │       │
5: ───────X───@───────@───
              │       │
6: ───────────X───@───X───
                  │
7: ───────────────Y───────,
 0: ───────────────────────@───
                          │
1: ───────────────────────@───
                          │
2: ───────────────────@───┼───
                      │   │
3: ───(0)───────(0)───┼───┼───
      │         │     │   │
4: ───┼─────────┼─────@───X───
      │         │     │
5: ───@─────────@─────X───────
      │         │
6: ───X─────@───X─────────────
            │
7: ─────────Y─────────────────,
 0: ───@─────────────────────────
      │
1: ───(0)───────────────────────
      │
2: ───┼─────@───────────────────
      │     │
3: ───┼─────┼───(0)───────(0)───
      │     │   │         │
4: ───X─────@───┼─────────┼─────
            │   │   

In [182]:
def cancel_double_T_gates(list_of_Q_circuits):
    # takes in list of circuits and simplifies repeated Toffoli gates 
    simplified_circuit_list=list_of_Q_circuits.copy()
    for index in range(len(simplified_circuit_list)-1):
        circuit_1_ops = list(simplified_circuit_list[index].all_operations())
        circuit_2_ops = list(simplified_circuit_list[index+1].all_operations())
        c1_new, c2_new = recursive_simplification(circuit_1_ops, circuit_2_ops)

        simplified_circuit_list[index] = cirq.Circuit(c1_new)
        simplified_circuit_list[index+1] = cirq.Circuit(c2_new)
    return simplified_circuit_list
simple_gate_cancel_list=cancel_double_T_gates(unsimplified_gate_seq)
simple_gate_cancel_list

[0: ───@───────────────────
      │
1: ───@───────────────────
      │
2: ───┼───@───────────────
      │   │
3: ───┼───┼───@───────@───
      │   │   │       │
4: ───X───@───┼───────┼───
          │   │       │
5: ───────X───@───────@───
              │       │
6: ───────────X───@───X───
                  │
7: ───────────────Y───────,
 0: ───────────────────────@───
                          │
1: ───────────────────────@───
                          │
2: ───────────────────@───┼───
                      │   │
3: ───(0)───────(0)───┼───┼───
      │         │     │   │
4: ───┼─────────┼─────@───X───
      │         │     │
5: ───@─────────@─────X───────
      │         │
6: ───X─────@───X─────────────
            │
7: ─────────Y─────────────────,
 0: ───@─────────────────────────
      │
1: ───(0)───────────────────────
      │
2: ───┼─────@───────────────────
      │     │
3: ───┼─────┼───(0)───────(0)───
      │     │   │         │
4: ───X─────@───┼─────────┼─────
            │   │   

In [642]:
def simplify_T_controls_differing_by_one_bit(list_of_Q_circuits):
    
    # simplifies Tofolli gates that differ by one bit
    """
        0: ───@─────@────        . 0: ───@───
              │     │            .       │    
        1: ───@────(0)───        . 1: ───│───
              │     │        ==> .       │   
        2: ───┼─────┼────        . 2: ───┼─── 
              │     │            .       │   
        3: ───┼─────┼────        . 3: ───┼─── 
              │     │            .       │   
        4: ───X─────X────        . 4: ───X─── 
                 
    """
    
    simplified_circuit_list=list_of_Q_circuits.copy()
    for index in range(len(simplified_circuit_list)-1):
        circuit_1_ops = list(simplified_circuit_list[index].all_operations())
        circuit_2_ops = list(simplified_circuit_list[index+1].all_operations())
        
        
        if (circuit_1_ops[-1].qubits==circuit_2_ops[0].qubits):
            cirq_1_control=np.array(circuit_1_ops[-1].control_values).reshape([len(circuit_1_ops[-1].control_values)])
            cirq_2_control= np.array(circuit_2_ops[0].control_values).reshape([len(circuit_1_ops[-1].control_values)])
            diff = cirq_1_control - cirq_2_control
            if np.abs(sum(diff))==1:
#                 print(cirq_1_control, cirq_2_control)
#                 print(index)
                index_to_keep = np.where(cirq_1_control==cirq_2_control)[0][0]
                active_qubits = circuit_1_ops[-1].qubits
#                 new_gate = cirq.CNOT(active_qubits[index_to_keep], active_qubits[-1])
            
                new_gate= cirq.Circuit(cirq.X.controlled(num_controls=1,
                                               control_values= [int(cirq_1_control[index_to_keep])] 
                                              ).on(active_qubits[index_to_keep], active_qubits[-1]))
            
                
                new_list=list(simplified_circuit_list[index].all_operations())
                new_list.pop(-1)
                new_list.append(new_gate)
                simplified_circuit_list[index]=cirq.Circuit(new_list)
                
                
                new2=list(simplified_circuit_list[index+1].all_operations())
                new2.pop(0)
                simplified_circuit_list[index+1]=cirq.Circuit(new2)
    return simplified_circuit_list
    
    
cancel_simple_T = simplify_T_controls_differing_by_one_bit(simple_gate_cancel_list)

In [643]:
ii=3

print(cancel_simple_T[ii])
print('')
print('')
print(cancel_simple_T[ii+1])

3: ───────@───────
          │
4: ───────┼───@───
          │   │
5: ───────@───X───
          │
6: ───@───X───────
      │
7: ───Y───────────


3: ───@───────────
      │
5: ───@───────@───
      │       │
6: ───X───@───X───
          │
7: ───────Y───────


In [332]:
list(cancel_simple_T[0].all_operations())[-1].qubits

qubits = set(list(cancel_simple_T[-1].all_operations())[-1].qubits, list(cancel_simple_T[0].all_operations())[-1].qubits)

TypeError: set expected at most 1 arguments, got 2

In [326]:
# list(unsimplified_gate_seq[0].all_operations())[-1].gate.sub_gate
list(cancel_simple_T[0].all_operations())[-1].qubits
list(unsimplified_gate_seq[0].all_operations())[-1].control_values

((1,), (1,))

In [341]:
# list(cancel_simple_T[0].all_operations())
# list(cancel_simple_T[1].all_operations())

list(cancel_simple_T[0].all_operations())[-2].qubits == list(cancel_simple_T[0].all_operations())

set((*list(cancel_simple_T[0].all_operations())[-2].qubits, 
     *list(cancel_simple_T[0].all_operations())[-1].qubits))

{cirq.LineQubit(5), cirq.LineQubit(6), cirq.LineQubit(7)}

In [390]:
list(cancel_simple_T[1].all_operations())[1].qubits

(cirq.LineQubit(3), cirq.LineQubit(5), cirq.LineQubit(6))

In [398]:
animals=['cat', 'dog', 'wolf', 'pig']
animals.pop(*[1,2])

TypeError: pop() takes at most 1 argument (2 given)

In [644]:
# def simplify_T_X_T(list_of_Q_circuits):
    
#     simplified_circuit_list=list_of_Q_circuits.copy()
#     for index in range(len(simplified_circuit_list)-1):
#         circuit_1_ops = list(simplified_circuit_list[index].all_operations())
#         circuit_2_ops = list(simplified_circuit_list[index+1].all_operations())
        
#         if (circuit_1_ops[-2].qubits==circuit_2_ops[0].qubits) and (circuit_1_ops[-1].gate.sub_gate == cirq.X):
#             if len(circuit_1_ops[-2].qubits)==3:
#                 if circuit_1_ops[-2].control_values==circuit_1_ops[-2].control_values:
#                     cirq_2_control= np.array(circuit_2_ops[0].control_values).reshape([len(circuit_1_ops[-2].control_values)])

#                     cnot_control_val = circuit_1_ops[-1].control_values[0][0]
#                     cnot_c_qubit = circuit_1_ops[-1].qubits[0]
#                     qubits = set((*circuit_1_ops[-1].qubits, *circuit_1_ops[-2].qubits))
#                     qubits.remove(circuit_1_ops[-1].qubits[0]) #remove CNOT qubit
#                     qubits.remove(circuit_1_ops[-1].qubits[1]) #remove CNOT qubit

#                     T_remaining_index = circuit_1_ops[-2].qubits.index(list(qubits)[0])
# #                     print(circuit_1_ops[-2].qubits[T_remaining_index])
# #                     print(circuit_1_ops[-2].control_values[T_remaining_index])
#                     print(circuit_1_ops[-1].qubits)
#                     print(cnot_c_qubit)
#                     print(circuit_1_ops[-2].qubits)
#                     print(circuit_1_ops[-2].control_values)
                    
#                     remaining_T_control_val= int(circuit_1_ops[-2].control_values[T_remaining_index][0])
                    
#                     new_gate= cirq.Circuit(cirq.X.controlled(num_controls=2,
#                                                control_values= [cnot_control_val, remaining_T_control_val] 
#                                               ).on(cnot_c_qubit, *qubits))
                    

   
                
#                     circuit_1_ops.pop(-2)
#                     circuit_1_ops.append(new_gate)
                
#                     circuit_2_ops.pop(0)

#                     simplified_circuit_list[index]=cirq.Circuit(circuit_1_ops)
#                     simplified_circuit_list[index+1]=cirq.Circuit(circuit_2_ops)

                
#     return simplified_circuit_list
    
    
# final_circ_list = simplify_T_X_T(cancel_simple_T)

In [645]:
def simplify_T_X_T(list_of_Q_circuits):
        # simplifies Tofolli NOT Toffoli gates where Tofolli gates have same controls
    """
    0: ───────(0)───────         .0: ───(0)───(0)───
              │                  .      │     │
    1: ───@───┼─────@───         .1: ───┼─────@─────
          │   │     │      ===>  .      │     │
    2: ───@───X─────@───         .2: ───X─────┼─────
          │         │            .            │
    3: ───X─────────X───         .3: ─────────X─────
    
    """
    simplified_circuit_list=list_of_Q_circuits.copy()
    for index in range(len(simplified_circuit_list)-1):
        circuit_1_ops = list(simplified_circuit_list[index].all_operations())
        circuit_2_ops = list(simplified_circuit_list[index+1].all_operations())
        
        if (circuit_1_ops[-2].qubits==circuit_2_ops[0].qubits) and (circuit_1_ops[-1].gate.sub_gate == cirq.X):
            if len(circuit_1_ops[-2].qubits)==3:
                if circuit_1_ops[-2].control_values==circuit_1_ops[-2].control_values:
                    cirq_2_control= np.array(circuit_2_ops[0].control_values).reshape([len(circuit_1_ops[-2].control_values)])

                    cnot_control_val = circuit_1_ops[-1].control_values[0][0]
                    cnot_c_qubit = circuit_1_ops[-1].qubits[0]
                    
                    qubit_to_remove_index=circuit_1_ops[-2].qubits.index(circuit_1_ops[-1].qubits[-1])
                    control_list= list(int(bit) for control in circuit_1_ops[-2].control_values for bit in control)
                    control_list.pop(qubit_to_remove_index)
                    
                    qubits = list(circuit_1_ops[-2].qubits)
                    qubits.pop(qubit_to_remove_index)
                    
                    new_gate= cirq.Circuit(cirq.X.controlled(num_controls=2,
                                               control_values= [cnot_control_val, *control_list] 
                                              ).on(cnot_c_qubit, *qubits))
                    

   
                
                    circuit_1_ops.pop(-2)
                    circuit_1_ops.append(new_gate)
                
                    circuit_2_ops.pop(0)

                    simplified_circuit_list[index]=cirq.Circuit(circuit_1_ops)
                    simplified_circuit_list[index+1]=cirq.Circuit(circuit_2_ops)

                
    return simplified_circuit_list
    
    
final_circ_list = simplify_T_X_T(cancel_simple_T)

In [647]:
from cirq.circuits import InsertStrategy

overall_circuit = cirq.Circuit()
for circuit in final_circ_list:
    overall_circuit.append(circuit.all_operations(), strategy=InsertStrategy.NEW)
overall_circuit

0: ───@───────────────────────@─────────@─────────────────────────────────────────────────────────────────────────────────────────────────(0)─────────(0)─────────────────────────────────────────────────────────────────────────────────────@───
      │                       │         │                                                                                                 │           │                                                                                       │
1: ───@───────────────────────┼─────────┼─────────────────────────────────────────(0)─────────(0)─────────────────────────────────────────┼───────────┼───────────────────────────────────────────@─────────@─────────────────────────────────@───
      │                       │         │                                         │           │                                           │           │                                           │         │                                 │
2: ───┼───@───────────────────┼─────────@─────────────────────────────────────────┼───────────(0)─────────────────────────────────────────┼───────────@───────────────────────────────────────────┼─────────(0)─────────────────────────(0)───┼───
      │   │                   │         │                                         │           │                                           │           │                                           │         │                           │     │
3: ───┼───┼───@───────────────┼───(0)───┼───(0)───────────────────@───────────────┼─────(0)───┼─────(0)───────────────────@───────────────┼─────(0)───┼─────(0)───────────────────@───────────────┼───(0)───┼─────(0)───────────────@───┼─────┼───
      │   │   │               │   │     │   │                     │               │     │     │     │                     │               │     │     │     │                     │               │   │     │     │                 │   │     │
4: ───X───@───┼───────────────X───┼─────┼───┼─────────────────@───@───────────────X─────┼─────┼─────┼─────────────────@───@───────────────X─────┼─────┼─────┼─────────────────@───@───────────────X───┼─────┼─────┼─────────────────┼───@─────X───
          │   │                   │     │   │                 │   │                     │     │     │                 │   │                     │     │     │                 │   │                   │     │     │                 │   │
5: ───────X───@───────@───────────@─────X───@─────────@───────X───┼───────@─────────────@─────X─────@─────────@───────X───┼───────@─────────────@─────X─────@─────────@───────X───┼───────@───────────@─────X─────@─────────@───────@───X─────────
              │       │           │         │         │           │       │             │           │         │           │       │             │           │         │           │       │           │           │         │       │
6: ───────────X───@───X───@───────X─────────X─────@───X───@───────X───@───X───@─────────X───────────X─────@───X───@───────X───@───X───@─────────X───────────X─────@───X───@───────X───@───X───@───────X───────────X─────@───X───@───X─────────────
                  │       │                       │       │           │       │                           │       │           │       │                           │       │           │       │                         │       │
7: ───────────────Y───────Y───────────────────────Y───────Y───────────Y───────Y───────────────────────────Y───────Y───────────Y───────Y───────────────────────────Y───────Y───────────Y───────Y─────────────────────────Y───────Y─────────────────

### OVERALL

In [651]:
# # BAD print
# overall_circuit = cirq.Circuit([circuit.all_operations() for circuit in final_circ_list])
# overall_circuit

In [655]:
N_ancilla= 3

unsimplified_gate_seq = control_circuit(N_ancilla)
simple_gate_cancel_list=cancel_double_T_gates(unsimplified_gate_seq)

cancel_simple_T = simplify_T_controls_differing_by_one_bit(simple_gate_cancel_list)

cancel_T_X_T = simplify_T_X_T(cancel_simple_T)

from cirq.circuits import InsertStrategy
overall_circuit = cirq.Circuit()
for circuit in cancel_T_X_T:
    overall_circuit.append(circuit.all_operations(), strategy=InsertStrategy.NEW)
overall_circuit

0: ───@───────────@───@───────────────────────────────────────(0)───(0)───────────────────────────────────@───
      │           │   │                                       │     │                                     │
1: ───@───────────┼───┼───────────────(0)───(0)───────────────┼─────┼─────────────────@───@───────────────@───
      │           │   │               │     │                 │     │                 │   │               │
2: ───┼───@───────┼───@───────────────┼─────(0)───────────────┼─────@─────────────────┼───(0)───────(0)───┼───
      │   │       │   │               │     │                 │     │                 │   │         │     │
3: ───X───@───────X───┼───────@───────X─────┼─────────@───────X─────┼─────────@───────X───┼─────────@─────X───
          │           │       │             │         │             │         │           │         │
4: ───────X───@───────X───@───X───@─────────X─────@───X───@─────────X─────@───X───@───────X─────@───X─────────
              │           │       │               │       │               │       │             │
5: ───────────Y───────────Y───────Y───────────────Y───────Y───────────────Y───────Y─────────────Y─────────────

In [606]:
# N_ancilla= 3

# unsimplified_gate_seq = control_circuit(N_ancilla)
# simple_gate_cancel_list=cancel_double_T_gates(unsimplified_gate_seq)

# cancel_simple_T = simplify_T_controls_differing_by_one_bit(simple_gate_cancel_list)

# # cancel_T_X_T = simplify_T_X_T(cancel_simple_T) ## <- missing this part for comparison!

# from cirq.circuits import InsertStrategy
# overall_circuit_no_X_T = cirq.Circuit()
# for circuit in cancel_simple_T:
#     overall_circuit.append(circuit.all_operations(), strategy=InsertStrategy.NEW)
# overall_circuit

In [656]:
from cirq.circuits import InsertStrategy
non_simplified = cirq.Circuit()
for circuit in unsimplified_gate_seq:
    non_simplified.append(circuit.all_operations(), strategy=InsertStrategy.NEW)
non_simplified

0: ───@───────────────@───@─────────────────@─────@─────────────────────@─────(0)───────────────────(0)───(0)───────────────(0)───(0)───────────────(0)───(0)───────────────────(0)───@───────────────────@───
      │               │   │                 │     │                     │     │                     │     │                 │     │                 │     │                     │     │                   │
1: ───@───────────────@───(0)───────────────(0)───(0)───────────────────(0)───(0)───────────────────(0)───(0)───────────────(0)───@─────────────────@─────@─────────────────────@─────@───────────────────@───
      │               │   │                 │     │                     │     │                     │     │                 │     │                 │     │                     │     │                   │
2: ───┼───@───────@───┼───┼─────@───────@───┼─────┼─────(0)───────(0)───┼─────┼─────(0)───────(0)───┼─────┼─────@───────@───┼─────┼─────@───────@───┼─────┼─────(0)───────(0)───┼─────┼───(0)───────(0)───┼───
      │   │       │   │   │     │       │   │     │     │         │     │     │     │         │     │     │     │       │   │     │     │       │   │     │     │         │     │     │   │         │     │
3: ───X───@───────@───X───X─────@───────@───X─────X─────@─────────@─────X─────X─────@─────────@─────X─────X─────@───────@───X─────X─────@───────@───X─────X─────@─────────@─────X─────X───@─────────@─────X───
          │       │             │       │               │         │                 │         │                 │       │               │       │               │         │               │         │
4: ───────X───@───X─────────────X───@───X───────────────X─────@───X─────────────────X─────@───X─────────────────X───@───X───────────────X───@───X───────────────X─────@───X───────────────X─────@───X─────────
              │                     │                         │                           │                         │                       │                         │                         │
5: ───────────Y─────────────────────Y─────────────────────────Y───────────────────────────Y─────────────────────────Y───────────────────────Y─────────────────────────Y─────────────────────────Y─────────────

In [654]:
np.array_equal(non_simplified.unitary(), overall_circuit.unitary())

True

In [ ]:
cirq.Circuit(cirq.X.controlled(num_controls=2,control_values= [1, 1]).on(qubits[1], qubits[2],  qubits[3]))

In [462]:
qubits = list(cirq.LineQubit.range(4))
circuit = cirq.Circuit(
cirq.Circuit(cirq.X.controlled(num_controls=2,control_values= [1, 1]).on(qubits[1], qubits[2],  qubits[3])),
cirq.Circuit(cirq.X.controlled(num_controls=1,control_values= [0]).on(qubits[0], qubits[2])),
cirq.Circuit(cirq.X.controlled(num_controls=2,control_values= [1, 1]).on(qubits[1], qubits[2],  qubits[3])) 
)
circuit

0: ───────(0)───────
          │
1: ───@───┼─────@───
      │   │     │
2: ───@───X─────@───
      │         │
3: ───X─────────X───

In [463]:
qubits = list(cirq.LineQubit.range(4))
circuit2 = cirq.Circuit(
cirq.Circuit(cirq.X.controlled(num_controls=1,control_values= [0]).on(qubits[0], qubits[2])),
cirq.Circuit(cirq.X.controlled(num_controls=2,control_values= [0, 1]).on(qubits[0], qubits[1],  qubits[3])) 
)
circuit2

0: ───(0)───(0)───
      │     │
1: ───┼─────@─────
      │     │
2: ───X─────┼─────
            │
3: ─────────X─────

In [464]:
np.array_equal(circuit.unitary(), circuit2.unitary())

True

In [ ]:
qubits = list(cirq.LineQubit.range(4))
circuit = cirq.Circuit(
cirq.Circuit(cirq.X.controlled(num_controls=3, control_values=[0,0,0]).on(qubits[0],qubits[1],qubits[2],qubits[3])),
cirq.Circuit(cirq.X.controlled(num_controls=3, control_values=[1,0,0]).on(qubits[0],qubits[1],qubits[2],qubits[3])),
cirq.Circuit(cirq.X.controlled(num_controls=3, control_values=[1,1,0]).on(qubits[0],qubits[1],qubits[2],qubits[3])),
cirq.Circuit(cirq.X.controlled(num_controls=3, control_values=[0,1,0]).on(qubits[0],qubits[1],qubits[2],qubits[3])),
cirq.Circuit(cirq.X.controlled(num_controls=3, control_values=[0,1,1]).on(qubits[0],qubits[1],qubits[2],qubits[3])),
cirq.Circuit(cirq.X.controlled(num_controls=3, control_values=[1,1,1]).on(qubits[0],qubits[1],qubits[2],qubits[3])),
cirq.Circuit(cirq.X.controlled(num_controls=3, control_values=[1,0,1]).on(qubits[0],qubits[1],qubits[2],qubits[3])),
cirq.Circuit(cirq.X.controlled(num_controls=3, control_values=[0,0,1]).on(qubits[0],qubits[1],qubits[2],qubits[3])),
)
circuit

In [ ]:
qubits = list(cirq.LineQubit.range(4))
circuit2 = cirq.Circuit(
    cirq.I.on(qubits[0]),
    cirq.I.on(qubits[1]),
    cirq.I.on(qubits[2]),
    cirq.X.on(qubits[3]),)
circuit2

In [ ]:
np.array_equal(circuit.unitary(), circuit2.unitary())

In [617]:
qubits = list(cirq.LineQubit.range(4))
circuit = cirq.Circuit(
cirq.Circuit(cirq.X.controlled(num_controls=3, control_values=[0,0,0]).on(qubits[0],qubits[1],qubits[2],qubits[3])),
cirq.Circuit(cirq.X.controlled(num_controls=3, control_values=[1,0,0]).on(qubits[0],qubits[1],qubits[2],qubits[3]))
)
circuit

0: ───(0)───@─────
      │     │
1: ───(0)───(0)───
      │     │
2: ───(0)───(0)───
      │     │
3: ───X─────X─────

In [618]:
qubits = list(cirq.LineQubit.range(4))
circuit2 = cirq.Circuit(
cirq.I.on(qubits[0]),
cirq.Circuit(cirq.X.controlled(num_controls=2, control_values=[0,0]).on(qubits[1],qubits[2],qubits[3])),
)
circuit2

0: ───I─────────

1: ───────(0)───
          │
2: ───────(0)───
          │
3: ───────X─────

In [619]:
np.array_equal(circuit.unitary(), circuit2.unitary())

True

## cannot do

In [622]:
from openfermion.ops import QubitOperator

In [623]:
QubitOperator('X0')*QubitOperator('Y0')

1j [Z0]

In [625]:
class phase_Pauli_gate(cirq.SingleQubitGate):
    """
    Description

    Args:
        theta (float): angle to rotate by in radians.
        number_control_qubits (int): number of control qubits
    """

    def __init__(self, phase, pauli_str):
        self.phase = phase
        self.pauli_str = pauli_str

    def _unitary_(self):
        if self.pauli_str == 'X':
            return cirq.X._unitary_()*self.phase
        elif self.pauli_str == 'Y':
            return cirq.Y._unitary_()*self.phase
        elif self.pauli_str == 'Z':
            return cirq.Z._unitary_()*self.phase
        elif self.pauli_str == 'I':
            return cirq.I._unitary_()*self.phase
        else:
            raise ValueError('Not a Pauli operation')

    def num_qubits(self):
        return 1

    def _circuit_diagram_info_(self, args):
        return str(self.phase) +'*'+self.pauli_str

    def __str__(self):
        return str(self.phase) +'*'+self.pauli_str

    def __repr__(self):
        return self.__str__()

In [626]:
QubitOperator('Y0',-1)*QubitOperator('Z0', 1j)

(1-0j) [X0]

In [628]:
qubits = list(cirq.LineQubit.range(4))
circuit = cirq.Circuit(
cirq.Circuit(cirq.X.controlled(num_controls=3, control_values=[0,0,0]).on(qubits[0],qubits[1],qubits[2],qubits[3])),
cirq.Circuit(cirq.Y.controlled(num_controls=3, control_values=[1,0,0]).on(qubits[0],qubits[1],qubits[2],qubits[3]))
)
circuit

0: ───(0)───@─────
      │     │
1: ───(0)───(0)───
      │     │
2: ───(0)───(0)───
      │     │
3: ───X─────Y─────

In [631]:
phase_Z0_gate = phase_Pauli_gate(1j, 'Z')
phase_Y_gate = phase_Pauli_gate(-1, 'Y')

qubits = list(cirq.LineQubit.range(4))
circuit2 = cirq.Circuit(
#     cirq.Circuit(phase_Y_gate.controlled(num_controls=1, control_values=[0]).on(qubits[0],qubits[3])),
#     cirq.Circuit(cirq.X.controlled(num_controls=1, control_values=[1]).on(qubits[0],qubits[3])),
    cirq.I.on(qubits[0]),
    cirq.Circuit(phase_Z0_gate.controlled(num_controls=2, control_values=[0,0]).on(qubits[1],qubits[2],qubits[3])),
)
circuit2

0: ───I──────────

1: ───────(0)────
          │
2: ───────(0)────
          │
3: ───────1j*Z───

In [632]:
np.array_equal(circuit.unitary(), circuit2.unitary())

False